In [1]:
# Quantum
import pennylane as qml
# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# Numpy, Pandas
import numpy as np
import pandas as pd
# Layer
from kan import KAN
from RNN_block import RNN_block
# Data processing
from fucntions_1028 import data_seq, train_seq
from sklearn.preprocessing import MinMaxScaler
# Quantum User-Def Classes
from utils import my_utils
from NQE_class import NQE
from NQE_train_class import NQE_Train

In [2]:
# CSV 파일 경로
feature_path = './data/pca/pca_train_data_Perth.csv'
label_path = './data/pca/pca_label_data_Perth.csv'

# CSV 파일을 읽어 Tensor로 변환
feature_data = torch.tensor(pd.read_csv(feature_path).values, dtype=torch.float32)
label_data = torch.tensor(pd.read_csv(label_path).values, dtype=torch.float32)

# data_seq 객체 생성
data = data_seq(feature_data, label_data)
train_loader, test_loader = data.split_data(test_ratio = 0.2, batch_size = 32, seq_first = False, for_nqe = True, seed = 40)


In [3]:
len(feature_data)

3193

In [4]:
for t, l in test_loader:
    print(l.shape)
    
    break

torch.Size([32, 2])


In [5]:
def criterion(pred, label):
    '''
        pred : inner product of two states
        label : label data
    '''
    print('pred : ', pred.shape)
    print('label :', label.shape)
    print('lbl[:, 0][-1] :', label[:, 0][-1])
    print('2nd term :', (label[:, 0] * label[:, 1] + 1).shape)
    loss = torch.sum(((pred) - 0.5 * (label[:, 0] * label[:, 1] + 1)) ** 2 ) / len(pred)
    return loss

def accuarcy(pred, label):
    '''
        pred : inner product of two states
        label : label data
    '''
    acc = torch.sum((torch.round(pred) == torch.round(0.5 * (label[:, 0] * label[:, 1] + 1)))) / len(pred)
    return acc

In [6]:
for t, l in train_loader:
    print(t.shape)
    break

torch.Size([32, 2, 4])


In [7]:
n = 4
test_nqe = NQE(n_feature=n, mode='FC')
test_train = NQE_Train(test_nqe, criterion, train_loader, test_loader, [accuarcy])
tested_nqe = test_train.train(5, seq_first=True)

===== 1 =====
pred :  torch.Size([32])
label : torch.Size([2, 32])
lbl[:, 0][-1] : tensor(1.)
2nd term : torch.Size([2])


RuntimeError: The size of tensor a (32) must match the size of tensor b (2) at non-singleton dimension 0

In [5]:
n = 4
fc_nqe = NQE(n_feature=n, mode='FC')
fc_nqe_train = NQE_Train(fc_nqe, criterion, train_loader, test_loader, [accuarcy])

In [7]:
trained_nqe = fc_nqe_train.train(1000, seq_first=True)

 loss : 0.10792 metric0 : 0.88889

 train_loss : 0.10792 test_loss : 0.15034 train_metric : 0.88889 test_metric : 0.81975

 loss : 0.14484 metric0 : 0.81481

 train_loss : 0.14484 test_loss : 0.15750 train_metric : 0.81481 test_metric : 0.80721

 loss : 0.15150 metric0 : 0.81481

 train_loss : 0.15150 test_loss : 0.15551 train_metric : 0.81481 test_metric : 0.80564

 loss : 0.15104 metric0 : 0.77778

 train_loss : 0.15104 test_loss : 0.16275 train_metric : 0.77778 test_metric : 0.78213

 loss : 0.19727 metric0 : 0.77778

 train_loss : 0.19727 test_loss : 0.15934 train_metric : 0.77778 test_metric : 0.81661

 loss : 0.10643 metric0 : 0.88889

 train_loss : 0.10643 test_loss : 0.17221 train_metric : 0.88889 test_metric : 0.76959

 loss : 0.08329 metric0 : 0.88889

 train_loss : 0.08329 test_loss : 0.15080 train_metric : 0.88889 test_metric : 0.80408

 loss : 0.13570 metric0 : 0.81481

 train_loss : 0.13570 test_loss : 0.15830 train_metric : 0.81481 test_metric : 0.81191

 loss : 0.11732 

In [82]:
trained_nqe_after_500 = fc_nqe_train.train(500, seq_first=True)

 loss : 0.05628 metric0 : 0.92593

 train_loss : 0.05628 test_loss : 0.18387 train_metric : 0.92593 test_metric : 0.76646

 loss : 0.05166 metric0 : 0.92593

 train_loss : 0.05166 test_loss : 0.18724 train_metric : 0.92593 test_metric : 0.76176

 loss : 0.08438 metric0 : 0.92593

 train_loss : 0.08438 test_loss : 0.19689 train_metric : 0.92593 test_metric : 0.75549

 loss : 0.18350 metric0 : 0.81481

 train_loss : 0.18350 test_loss : 0.18451 train_metric : 0.81481 test_metric : 0.75862

 loss : 0.02040 metric0 : 1.00000

 train_loss : 0.02040 test_loss : 0.18872 train_metric : 1.00000 test_metric : 0.76176

 loss : 0.04230 metric0 : 0.96296

 train_loss : 0.04230 test_loss : 0.17951 train_metric : 0.96296 test_metric : 0.77743

 loss : 0.20455 metric0 : 0.77778

 train_loss : 0.20455 test_loss : 0.18455 train_metric : 0.77778 test_metric : 0.76332

 loss : 0.22920 metric0 : 0.74074

 train_loss : 0.22920 test_loss : 0.19237 train_metric : 0.74074 test_metric : 0.76646

 loss : 0.02057 

In [8]:
torch.save(trained_nqe.state_dict(), "./models/pca_FC_NQE_loss187.pth")

### 실제로 같은 label을 갖는 feature에 대해서는 Embedding한 state간의 내적이 1에 가깝고, 그렇지 않으면 0으로 되는지 확인

In [70]:
for e in train_loader:
    print(e)
    break

[tensor([[[-1.2158e+00,  2.6049e+00,  3.2846e-01,  6.1858e-01],
         [-1.4919e+00,  7.5409e-01, -9.8366e-02, -3.3946e-01]],

        [[ 1.8011e+00,  7.8165e-01, -2.7626e-01, -7.9209e-01],
         [-2.8622e+00, -8.9282e-01,  9.9488e-02,  4.8230e-02]],

        [[-1.3783e+00, -5.2450e-01, -2.6319e-01, -4.1469e-01],
         [-2.4822e+00, -9.5507e-01, -5.0605e-01, -2.8460e-03]],

        [[ 5.7335e-01,  9.6609e-02, -3.2476e-01, -5.8291e-01],
         [ 2.1940e-01, -8.3477e-02, -2.9614e-03, -8.8602e-01]],

        [[-2.2716e+00, -1.1708e+00, -1.1284e+00,  4.3273e-01],
         [-1.7903e+00,  4.0601e+00, -2.1713e-01,  3.1490e+00]],

        [[-1.2252e+00, -1.2273e+00,  9.5935e-01, -1.1580e-01],
         [ 3.1160e+00, -1.5854e+00,  6.1983e-02, -1.7014e-01]],

        [[ 1.7007e+00, -7.2836e-01,  1.0045e+00,  8.3188e-02],
         [-1.3481e+00, -1.7277e+00,  9.1373e-01,  8.5572e-01]],

        [[ 1.9020e+00,  1.9909e+00,  6.9364e-01, -5.0573e-01],
         [ 2.1197e+00,  1.2863e+00,  7.7

In [75]:
for e in train_loader:
    f1, l1 = e[0][0][0], e[1][0][0]
    f2, l2 = e[0][0][1], e[1][0][1]
    f3, l3 = e[0][1][1], e[1][1][1]
    f4, l4 = e[0][2][1], e[1][2][1]
    
    print(f1, l1)
    print(f2, l2)
    print(f3, l3)
    print(f4, l4)
    break
# print(torch.stack([f1, f2]))
f_data = torch.stack([torch.stack([f1, f2]), torch.stack([f2, f3]), torch.stack([f3, f1]), torch.stack([f3, f4]), torch.stack([f2, f4])])
l_data = torch.stack([torch.stack([l1, l2]), torch.stack([l2, l3]), torch.stack([l3, l1]), torch.stack([l3, l4]), torch.stack([l2, l4])])

# print(f_data, f_data.shape)
# print(f_data.permute(1, 0, 2), f_data.permute(1,0,2).shape)


tensor([ 4.4700, -0.5930,  0.0424,  0.4030]) tensor(-1.)
tensor([ 2.7916, -0.4528, -1.0187, -0.6341]) tensor(-1.)
tensor([ 1.1479, -2.6952,  0.1751,  0.9549]) tensor(-1.)
tensor([-1.1129,  1.7565, -1.9460,  0.1122]) tensor(1.)


In [76]:
print(f_data)
print(l_data)
trained_nqe(f_data.permute(1, 0, 2))

tensor([[[ 4.4700, -0.5930,  0.0424,  0.4030],
         [ 2.7916, -0.4528, -1.0187, -0.6341]],

        [[ 2.7916, -0.4528, -1.0187, -0.6341],
         [ 1.1479, -2.6952,  0.1751,  0.9549]],

        [[ 1.1479, -2.6952,  0.1751,  0.9549],
         [ 4.4700, -0.5930,  0.0424,  0.4030]],

        [[ 1.1479, -2.6952,  0.1751,  0.9549],
         [-1.1129,  1.7565, -1.9460,  0.1122]],

        [[ 2.7916, -0.4528, -1.0187, -0.6341],
         [-1.1129,  1.7565, -1.9460,  0.1122]]])
tensor([[-1., -1.],
        [-1., -1.],
        [-1., -1.],
        [-1.,  1.],
        [-1.,  1.]])


tensor([0.9924, 1.0000, 0.9924, 0.0569, 0.0569], dtype=torch.float64,
       grad_fn=<SelectBackward0>)

### KAN NQE도?

In [77]:
kan_nqe = NQE(n_feature=4, mode='KAN')
kan_nqe_train = NQE_Train(kan_nqe, criterion, train_loader, test_loader, [accuarcy])

checkpoint directory created: ./model
saving model version 0.0


In [79]:
trained_kanqe = kan_nqe_train.train(100, seq_first=True)

 loss : 0.73604 metric0 : 0.18519

 train_loss : 0.73604 test_loss : 0.58877 train_metric : 0.18519 test_metric : 0.34326

 loss : 0.60606 metric0 : 0.29630

 train_loss : 0.60606 test_loss : 0.58245 train_metric : 0.29630 test_metric : 0.34169

 loss : 0.64014 metric0 : 0.29630

 train_loss : 0.64014 test_loss : 0.58127 train_metric : 0.29630 test_metric : 0.34483

 loss : 0.67932 metric0 : 0.25926

 train_loss : 0.67932 test_loss : 0.58545 train_metric : 0.25926 test_metric : 0.34169

 loss : 0.58306 metric0 : 0.33333

 train_loss : 0.58306 test_loss : 0.59054 train_metric : 0.33333 test_metric : 0.34326

 loss : 0.70631 metric0 : 0.22222

 train_loss : 0.70631 test_loss : 0.59048 train_metric : 0.22222 test_metric : 0.34169

 loss : 0.68561 metric0 : 0.22222

 train_loss : 0.68561 test_loss : 0.59146 train_metric : 0.22222 test_metric : 0.34326

 loss : 0.52930 metric0 : 0.40741

 train_loss : 0.52930 test_loss : 0.59612 train_metric : 0.40741 test_metric : 0.34013

 loss : 0.64713 

In [80]:
print(f_data)
print(l_data)
trained_kanqe(f_data.permute(1, 0, 2))

tensor([[[ 4.4700, -0.5930,  0.0424,  0.4030],
         [ 2.7916, -0.4528, -1.0187, -0.6341]],

        [[ 2.7916, -0.4528, -1.0187, -0.6341],
         [ 1.1479, -2.6952,  0.1751,  0.9549]],

        [[ 1.1479, -2.6952,  0.1751,  0.9549],
         [ 4.4700, -0.5930,  0.0424,  0.4030]],

        [[ 1.1479, -2.6952,  0.1751,  0.9549],
         [-1.1129,  1.7565, -1.9460,  0.1122]],

        [[ 2.7916, -0.4528, -1.0187, -0.6341],
         [-1.1129,  1.7565, -1.9460,  0.1122]]])
tensor([[-1., -1.],
        [-1., -1.],
        [-1., -1.],
        [-1.,  1.],
        [-1.,  1.]])


tensor([0.0201, 0.1455, 0.1010, 0.1053, 0.0003], dtype=torch.float64,
       grad_fn=<SelectBackward0>)

---

### RNNQE

In [85]:
from fucntions_1029 import data_seq, train_seq
from RNNQE_class import NQE

In [87]:
rnnqe = NQE(n_feature=4, mode='RNN', rnn_sequence=5)